In [1]:
# imports
# system related
import sys
import subprocess

# basics
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep

# snap7 and logging
import snap7 as s7


print('imports succesful')

imports succesful


## Connecting to plc

In [2]:
# connecting to plc

plc = s7.client.Client()
plc.connect('192.168.0.19', 0, 1, )  # ip, rack, slot

print(f'connection status: {plc.get_connected()}')

b' TCP : Unreachable peer'


RuntimeError: b' TCP : Unreachable peer'

## Defining get & set functions for plc variables

In [8]:
# Defining read/write functions

def get_bool(db_number, start_offset, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    value = s7.util.get_bool(data, 0, bit_offset)
    # print(f'db number: {db_number}, bit: {start_offset}, has boolean value: {value}')
    return value
    
def set_bool(db_number, start_offset, value, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    s7.util.set_bool(data, 0, bit_offset, value)
    plc.db_write(db_number, start_offset, data)
    return None


def get_real(db_number, start_offset):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    real = s7.util.get_real(data, 0)
    return real

def set_real(db_number, start_offset, value):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    s7.util.set_real(data, 0, value)
    plc.db_write(db_number, start_offset, data)
    return None



    


## Creating RPMs to loop thru

In [9]:
# creating random values to loop thru

############ CHECK THESE VAls!!

min_rpm = 22
max_rpm = 37
d_rpm =   1
num_values = 1111

rpm_interval = 5  # s


rpms = [min_rpm]


while len(rpms) < num_values:
    # create random rpm value
    rpm = int(np.round(np.random.uniform(min_rpm, max_rpm),0))
    # append if difference is high enough. 
    if abs(rpm-rpms[-1]) >= d_rpm:
        rpms.append(rpm)

print_cols = 50
print(f'RPMs to loop thru, in pairs of {print_cols}:\n') 

print(f'  {1} to  {print_cols+1}| ', end='')
for i, val in enumerate(rpms):
    if (i+1)%print_cols != 0:
        print(val, end=' ')
    else: 
        if i == 49:
            print(val, end =f'\n {i+1} to {i+print_cols+1}| ')
        else:
            if i+1 == num_values:
                print(val)
            else:
                print(val, end =f'\n{i+1} to {i+print_cols+1}| ')
print(f'\nExpected run time: {len(rpms)*rpm_interval/60} minutes')


RPMs to loop thru, in pairs of 50:

  1 to  51| 22 33 24 35 32 23 34 26 23 32 33 24 33 28 24 25 24 30 22 35 26 27 29 30 23 24 26 29 23 34 35 25 37 24 25 26 23 34 28 30 32 26 27 34 32 26 36 33 27 33
 50 to 100| 31 29 34 33 37 35 23 24 32 37 35 23 24 23 25 34 32 36 31 25 36 28 37 31 24 29 27 30 28 27 26 36 23 24 30 31 26 33 32 29 23 31 35 36 26 36 37 30 36 34
100 to 150| 29 32 37 34 32 31 28 25 31 27 34 35 24 33 36 32 31 35 27 33 35 34 37 26 37 35 31 29 31 34 36 30 31 22 29 22 24 25 32 26 27 33 30 32 33 31 24 34 35 33
150 to 200| 30 35 36 31 26 27 25 26 29 24 26 28 32 27 35 26 24 27 33 24 27 26 28 32 23 24 26 36 28 30 25 22 37 27 24 36 24 35 26 23 36 33 36 24 32 27 34 28 24 29
200 to 250| 25 29 22 25 26 35 31 24 31 33 31 23 29 35 34 35 34 30 37 34 36 37 32 33 31 29 35 31 37 26 33 27 30 37 32 23 27 26 31 35 24 35 23 37 25 24 34 35 23 34
250 to 300| 27 28 30 32 36 33 36 26 22 32 27 33 32 35 25 37 34 32 24 34 32 28 23 30 35 33 34 28 32 23 24 29 28 37 26 30 37 25 24 31 28 32 35 31 34 26 37 3

## Defining DB number and start offsets for the variables

In [10]:
db_py = 50

# defining start_offsets (so) for the variables in the DB50
so_rpm = 0
so_hz1 = 4
so_hz2 = 8
so_hz3 = 12
so_hz4 = 16


In [11]:
# for loop true random rpms

try:
    for i, rpm in enumerate(rpms):
        set_real(db_py, so_rpm, rpm)
        print(f'Progress: |{int(100*(i+1)/len(rpms))*"█" + int((100*(1-(i+1)/len(rpms))))*"-"}| {np.round((i+1)/len(rpms)*100 ,1)}% |{i+1}/{len(rpms)} | ETA: {np.round((len(rpms)-i)*rpm_interval/60,1)} min | RPM: {get_real(db_py, so_rpm)}', end='\r')
        sleep(rpm_interval)
        
except KeyboardInterrupt:
    print('Sending RPMs interupted by keystroke                                                         It came this far: ')

print('all rpms have been sent, the job is finished')

b' ISO : An error occurred during send TCP : Connection timed out'


RuntimeError: b' ISO : An error occurred during send TCP : Connection timed out'

In [6]:
# start make datalog function for ixon db

In [7]:
up_down = [min_rpm, max_rpm] * 150

print(up_down)

[22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37, 22, 37,

In [96]:
try:
    for i, rpm in enumerate(up_down):
        set_real(db_py, so_rpm, rpm)
        print(f'Progress: |{int(100*(i+1)/len(up_down))*"█" + int((100*(1-(i+1)/len(up_down))))*"-"}| {np.round((i+1)/len(up_down)*100 ,1)}% |{i+1}/{len(up_down)} | ETA: {np.round((len(up_down)-i)*rpm_interval/60,1)} min | RPM: {get_real(db_py, so_rpm)}', end='\r')
        sleep(rpm_interval)
        
except KeyboardInterrupt:
    print('Sending RPMs interupted by keystroke                                                         It came this far: ')


print('job finished')


Sending RPMs interupted by keystroke                                                         It came this far: | 19.0% |57/300 | ETA: 20.3 min | RPM: 22.0


In [6]:
1

1